# Cuery usage example

In [1]:
import importlib
from pathlib import Path

import instructor
import openai
import pandas as pd

from cuery import pprint, response, task, utils
from cuery.work import CountrySectors, DirceJobs, DirceTasks, SpanishSectors

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

In [2]:
task.Task.registry

{'SpanishSectors': <cuery.task.Task at 0x10d0fec10>,
 'CountrySectors': <cuery.task.Task at 0x118fc7b10>,
 'DirceJobs': <cuery.task.Task at 0x118fd8d50>,
 'DirceTasks': <cuery.task.Task at 0x118ff4410>}

# Starting without input context
"Pure prompt"

In [2]:
context = [{"country": "Unites States"}, {"country": "Spain"}]
sectors = await CountrySectors(context, n_concurrent=20)
sectors.to_pandas()

Gathering responses: 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]


,country,sector,subsector,sector_automation_potential
0,Unites States,Retail Trade,Furniture and Home Furnishings Stores,9
1,Unites States,Transportation and Warehousing,Truck Transportation,8
2,Unites States,Health Care and Social Assistance,Offices of Physicians,7
3,Unites States,Manufacturing,Food Manufacturing,8
4,Unites States,Accommodation and Food Services,Restaurants and Other Eating Places,9
5,Spain,Manufacturing,Chemical Manufacturing,8
6,Spain,Retail Trade,Clothing and Clothing Accessories Stores,7
7,Spain,Health Care and Social Assistance,Ambulatory Health Care Services,9
8,Spain,Accommodation and Food Services,Full-Service Restaurants,7
9,Spain,Information,"Data Processing, Hosting, and Related Services",9


In [5]:
chain = task.Chain(SpanishSectors, DirceJobs, DirceTasks)
result = await chain(model="gpt-4.1-mini", n_concurrent=20)
result

[05/22/25 19:12:51] INFO     [1/3] Running task 'Sectors'                                               ]8;id=189719;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=780658;file:///Users/thomas/code/cuery/src/cuery/task.py#183\183]8;;\

[05/22/25 19:12:54] INFO     [2/3] Running task 'Jobs'                                                  ]8;id=13634;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=801949;file:///Users/thomas/code/cuery/src/cuery/task.py#183\183]8;;\

Gathering responses: 100%|██████████| 5/5 [00:04<00:00,  1.15it/s]


[05/22/25 19:12:58] INFO     [3/3] Running task 'JobTasks'                                              ]8;id=673156;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=83148;file:///Users/thomas/code/cuery/src/cuery/task.py#183\183]8;;\

Gathering responses: 100%|██████████| 15/15 [00:09<00:00,  1.59it/s]


,sector,subsector,job_role,task,task_description,task_automation_potential,intelligence,sexyness,scalability,data_needs,products
0,Administrative and Support and Waste Managemen...,Employment Services,Recruitment Consultant,Resume Screening,Automatically analyze and filter resumes based...,9,6,5,8,Large datasets of resumes and job descriptions...,"[HireVue, Pymetrics, ZipRecruiter]"
1,Administrative and Support and Waste Managemen...,Employment Services,Recruitment Consultant,Candidate Matching,Match candidates to job openings using AI to i...,9,7,6,9,Extensive candidate profiles and job requireme...,"[LinkedIn Talent Solutions, Eightfold AI, Hire..."
2,Administrative and Support and Waste Managemen...,Employment Services,Recruitment Consultant,Interview Scheduling,Automate scheduling of interviews through AI a...,8,4,7,9,Access to calendars and candidate and recruite...,"[Calendly, x.ai, Clara Labs]"
3,Administrative and Support and Waste Managemen...,Employment Services,Recruitment Consultant,Candidate Communication,Use AI chatbots to handle routine candidate co...,8,5,6,8,Candidate query logs and frequently asked ques...,"[Mya, Olivia by Paradox, XOR.ai]"
4,Administrative and Support and Waste Managemen...,Employment Services,Recruitment Consultant,Job Description Generation,Generate or optimize job descriptions using na...,7,6,6,7,Job role data and examples of effective job de...,"[Textio, Jobpal, Copy.ai]"
...,...,...,...,...,...,...,...,...,...,...,...
71,Transportation and Warehousing,Couriers and Express Delivery Services,Customer Support Representative,Responding to common customer inquiries,Automatically answering frequently asked quest...,9,5,5,9,"Customer inquiry templates, FAQ database, cust...","[Zendesk Answer Bot, Freshdesk Freddy AI, Inte..."
72,Transportation and Warehousing,Couriers and Express Delivery Services,Customer Support Representative,Order status updates,Providing customers with real-time updates on ...,10,4,6,10,"Real-time delivery tracking data, order inform...","[AfterShip, Parcel Pending, Shippo]"
73,Transportation and Warehousing,Couriers and Express Delivery Services,Customer Support Representative,Automated ticket routing,Automatically categorizing and routing custome...,8,7,4,8,"Historical ticket data, department categories,...","[Zendesk, Freshdesk, Salesforce Service Cloud]"
74,Transportation and Warehousing,Couriers and Express Delivery Services,Customer Support Representative,Generating customer support reports,Using AI to compile and generate reports on cu...,7,6,3,7,"Customer interaction logs, support metrics data","[Power BI, Tableau, Zoho Analytics]"


In [8]:
display(chain._usage)
chain._usage[["prompt", "completion", "cost"]].sum()

,prompt,completion,cost,task_index,task
0,313,127,0.01701,0,Sectors
0,434,156,0.02238,1,Jobs
1,428,156,0.02220,1,Jobs
2,430,144,0.02154,1,Jobs
3,431,171,0.02319,1,Jobs
4,432,148,0.02184,1,Jobs
0,551,504,0.04677,2,JobTasks
1,552,350,0.03756,2,JobTasks
2,551,322,0.03585,2,JobTasks
3,546,376,0.03894,2,JobTasks


prompt        10705.00000
completion     6529.00000
cost              0.71289
dtype: float64

In [4]:
chain = task.Chain(CountrySectors, DirceJobs, DirceTasks)
result = await chain({"country": "Spain"}, model="gpt-4.1-mini", n_concurrent=20)
result

[05/22/25 14:15:02] INFO     [1/3] Running task 'Sectors'                                               ]8;id=79468;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=388627;file:///Users/thomas/code/cuery/src/cuery/task.py#202\202]8;;\

[05/22/25 14:15:05] INFO     [2/3] Running task 'Jobs'                                                  ]8;id=126949;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=519815;file:///Users/thomas/code/cuery/src/cuery/task.py#202\202]8;;\

Gathering responses: 100%|██████████| 5/5 [00:04<00:00,  1.22it/s]


[05/22/25 14:15:09] INFO     [3/3] Running task 'JobTasks'                                              ]8;id=654887;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=94567;file:///Users/thomas/code/cuery/src/cuery/task.py#202\202]8;;\

Gathering responses: 100%|██████████| 15/15 [00:16<00:00,  1.07s/it]


[05/22/25 14:15:25] WARNING  Encountered: 1 response parsing errors!                                    ]8;id=15192;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=642241;file:///Users/thomas/code/cuery/src/cuery/task.py#120\120]8;;\

,sector,subsector,job_role,task,task_description,task_automation_potential,intelligence,sexyness,scalability,data_needs,products
0,Manufacturing,Automobile and Electronics Manufacturing,Data Entry Clerk,Invoice Data Entry,Automatically input invoice details into digit...,9,3,4,8,Structured invoice data from scanned documents...,"[ABBYY FlexiCapture, UiPath, Kofax]"
1,Manufacturing,Automobile and Electronics Manufacturing,Data Entry Clerk,Inventory Records Updating,Automate updating materials and parts inventor...,8,4,5,7,Digital or scanned purchase orders and invento...,"[SAP Intelligent Robotic Process Automation, B..."
2,Manufacturing,Automobile and Electronics Manufacturing,Data Entry Clerk,Quality Control Data Logging,Input quality control test results and inspect...,7,5,5,6,Test results data from digital or scanned repo...,"[Siemens Teamcenter, Microsoft Power Automate,..."
3,Manufacturing,Automobile and Electronics Manufacturing,Data Entry Clerk,Employee Attendance and Timesheet Entry,Automate the entry of employee attendance and ...,8,3,3,8,Attendance logs and timesheets in digital or h...,"[Kronos Workforce Ready, ADP, SAP SuccessFactors]"
4,Manufacturing,Automobile and Electronics Manufacturing,Data Entry Clerk,Purchase Order Processing,Automate capturing and entering purchase order...,9,4,5,8,Digital or scanned purchase orders with standa...,"[Coupa, Oracle NetSuite, Basware]"
...,...,...,...,...,...,...,...,...,...,...,...
72,Transportation and Warehousing,Logistics and Freight Services,Logistics Coordinator,Inventory Management and Tracking,Tracking inventory levels and locations using ...,9,6,5,8,Real-time inventory data and location tracking...,"[Fishbowl Inventory, Zoho Inventory, NetSuite]"
73,Transportation and Warehousing,Logistics and Freight Services,Logistics Coordinator,Route Optimization,Optimizing delivery routes for efficiency and ...,9,7,7,9,"Data on routes, traffic, delivery locations, a...","[Route4Me, OptimoRoute, WorkWave Route Manager]"
74,Transportation and Warehousing,Logistics and Freight Services,Logistics Coordinator,Order Processing,"Automating order entry, validation, and proces...",8,5,6,8,"Order details, customer information, and inven...","[SAP Logistics, Oracle SCM, Infor Supply Chain]"
75,Transportation and Warehousing,Logistics and Freight Services,Logistics Coordinator,Documentation Generation,Generating shipping and customs documentation ...,7,4,4,7,"Data on shipments, customs forms, and regulato...","[DocuPhase, TransDoc, Kofax]"


# Jobs

In [10]:
DATA_DIR = GDRIVE / "Research/future_of_work/inputs/ine_dirce_aggregated_by_activity.csv"
data = pd.read_csv(DATA_DIR)
data.columns = [c.replace(" ", "_") for c in data.columns]
data = data.rename(columns={"Division": "sector", "Actividad_principal": "subsector"})
display(data)

n_examples = 5
df = data.iloc[:5]

,sector,subsector,Estimated_Employees_2024,Median_YoY_Growth_pct
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,134055,-2.4
1,"29 Fabricación de vehículos de motor, remolque...","293 Fabricación de componentes, piezas y acces...",97484,-0.2
2,43 Actividades de construcción especializada,"432 Instalaciones eléctricas, de fontanería y ...",303430,0.5
3,45 Venta y reparación de vehículos de motor y ...,451 Venta de vehículos de motor,102172,-0.5
4,46 Comercio al por mayor e intermediarios del ...,463 Comercio al por mayor de productos aliment...,325970,-0.9
5,46 Comercio al por mayor e intermediarios del ...,464 Comercio al por mayor de artículos de uso ...,228324,-2.0
6,"47 Comercio al por menor, excepto de vehículos...",471 Comercio al por menor en establecimientos ...,233708,1.3
7,"47 Comercio al por menor, excepto de vehículos...",477 Comercio al por menor de otros artículos e...,242341,-1.3
8,49 Transporte terrestre y por tubería,493 Otro transporte terrestre de pasajeros,142413,0.5
9,49 Transporte terrestre y por tubería,494 Transporte de mercancías por carretera y s...,273002,-1.5


In [7]:
pprint(DirceJobs)

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭────────────────────────────────────────────────── Prompt ───────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │  Required: ['Division', 'Actividad_principal']                                                              │ │
│ │                                                                                                             │ │
│ │ ╭──────────────────────────────────────────────── SYSTEM ─────────────────────────────────────────────────╮ │ │
│ │ │                                                                                                         │ │ │
│ │ │  You're an analyst at the Spanish 'Instituo Nacional de Estadística' (INE) analyzing data from its      │ │ │
│ │ │  'Directorio Central de Empresas' (DIRCE). Your objective is to analyze groups of companies, identifie  │ │ │
│ │ │  by a sector ('Division') and a corresponding main activity ('Actividad principal') in order to identi  │ │ │
│ │ │  jobs within those companies that are likely to be automatable by AI. Both 'Division' and               │ │ │
│ │ │  'Actividad_principal' are provided in Spanish and may include numeric IDs that you can ignore if you   │ │ │
│ │ │  don't understand them. Always respond in English. Only consider jobs that are computer- or paper-base  │ │ │
│ │ │  and can be automated by AI using software (don't include jobs automatable by robots or other physical  │ │ │
│ │ │  means).                                                                                                │ │ │
│ │ │                                                                                                         │ │ │
│ │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │ │
│ │ ╭───────────────────────────────────────────────── USER ──────────────────────────────────────────────────╮ │ │
│ │ │                                                                                                         │ │ │
│ │ │  Please analyze the following jobs sector and identify jobs that are automatable by AI software.        │ │ │
│ │ │  Division: {{Division}}                                                                                 │ │ │
│ │ │  Actividad principal: {{Actividad_principal}}                                                           │ │ │
│ │ │                                                                                                         │ │ │
│ │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│ ╭────────────────────────────────────────────── RESPONSE: Jobs ───────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │ ╭─ jobs: list[cuery.work.models.Job] ─────────────────────────────────────────────────────────────────────╮ │ │
│ │ │                                                                                                         │ │ │
│ │ │  A list of jobs with their AI automation potential and reasons for that potential                       │ │ │
│ │ │                                                                                                         │ │ │
│ │ │  {'required': True, 'metadata': [MinLen(min_length=3)]}                                                 │ │ │
│ │ │                                                   

## Task chain

In [11]:
chain = task.Chain(DirceJobs, DirceTasks)
result = await chain(data, model="gpt-4.1-mini", n_concurrent=20)
result

[05/23/25 10:48:31] INFO     [1/2] Running task 'Jobs'                                                  ]8;id=510080;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=240576;file:///Users/thomas/code/cuery/src/cuery/task.py#183\183]8;;\

Gathering responses: 100%|██████████| 23/23 [00:06<00:00,  3.32it/s]


[05/23/25 10:48:38] INFO     [2/2] Running task 'JobTasks'                                              ]8;id=208577;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=135507;file:///Users/thomas/code/cuery/src/cuery/task.py#183\183]8;;\

Gathering responses: 100%|██████████| 69/69 [00:27<00:00,  2.53it/s]


[05/23/25 10:49:05] WARNING  Encountered: 1 response parsing errors!                                    ]8;id=307948;file:///Users/thomas/code/cuery/src/cuery/task.py\task.py]8;;\:]8;id=422555;file:///Users/thomas/code/cuery/src/cuery/task.py#131\131]8;;\

,sector,subsector,job_role,task,task_description,task_automation_potential,intelligence,sexyness,scalability,data_needs,products
0,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Quality Control Analyst,Data Entry and Report Generation,Automate entering quality control data and gen...,8,5,4,9,"Quality inspection data, test results, report ...","[AutoEntry, Microsoft Power Automate, UiPath]"
1,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Quality Control Analyst,Statistical Analysis,Perform automated statistical analysis of qual...,7,7,5,8,"Historical quality data, control charts, proce...","[Minitab, JMP, SPSS]"
2,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Quality Control Analyst,Document Control and Compliance Tracking,Automate monitoring and updating compliance re...,8,6,3,8,"Compliance checklists, certificates, document ...","[DocuWare, ConvergePoint, PaperTracer]"
3,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Quality Control Analyst,Sampling Schedule Management,Automate scheduling and reminders for routine ...,7,4,6,7,"Sampling schedules, testing frequency data, ca...","[Schedly, When I Work, Deputy]"
4,10 Industria de la alimentación,101 Procesado y conservación de carne y elabor...,Quality Control Analyst,Quality Trend Reporting,Create automated reports highlighting trend ch...,7,6,5,8,"Time series quality data, production logs, rep...","[Tableau, Power BI, Qlik Sense]"
...,...,...,...,...,...,...,...,...,...,...,...
342,85 Educación,855 Otra educación,Curriculum Research Assistant,Literature Review Summarization,Automatically summarizing research articles an...,8,7,6,9,Access to digital research articles and educat...,"[Zamzar AI summarizer, Scholarcy]"
343,85 Educación,855 Otra educación,Curriculum Research Assistant,Data Collection and Organization,Gathering and structuring data from various ed...,9,6,5,8,Databases of educational surveys and research ...,"[Microsoft Power Automate, Zapier]"
344,85 Educación,855 Otra educación,Curriculum Research Assistant,Trend Analysis in Education,Analyzing trends from research data and educat...,7,7,6,7,Historical and current educational trend data.,"[Tableau, Google Data Studio]"
345,85 Educación,855 Otra educación,Curriculum Research Assistant,Report Generation,Automatically generating structured reports an...,8,6,7,8,Formatted research data and analysis results.,"[Grammarly Business, Jasper AI]"


In [13]:
chain._usage[["prompt", "completion", "cost"]].sum()

prompt        50067.00000
completion    29686.00000
cost              3.28317
dtype: float64

In [14]:
result.to_csv(
    GDRIVE / "Research/future_of_work/outputs/ine_dirce_ai_tasks.csv",
    index=False,
)

## Test LiteLLM

In [7]:
...

Ellipsis

## Test Perplexity

In [ ]:
# jobs with Perplexity (using its OpenAI-compatible API)
import os

utils.set_api_keys()

perplex = openai.AsyncOpenAI(
    api_key=os.getenv("PERPLEXITY_API_KEY"), base_url="https://api.perplexity.ai"
)
perplex = instructor.from_perplexity(perplex)

jobs_perp = await DirceJobs(df, client=perplex, model="sonar", n_concurrent=2)

Gathering responses: 100%|██████████| 5/5 [06:37<00:00, 79.58s/it] 

Encountered: 5 response parsing errors!
